In [1]:
#select any realife dataset and apply CNN
!pip install kaggle wandb

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [3]:
!kaggle datasets download -d harshvardhan21/sign-language-detection-using-images

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/__init__.py", line 3, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/__init__.py", line 6, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 258, in <module>
    class KaggleApi(KaggleApi):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 289, in KaggleApi
    os.makedirs(config_dir)
  File "<frozen os>", line 225, in makedirs
OSError: [Errno 30] Read-only file system: '/content'


In [4]:
!unzip sign-language-detection-using-images.zip

unzip:  cannot find or open sign-language-detection-using-images.zip, sign-language-detection-using-images.zip.zip or sign-language-detection-using-images.zip.ZIP.


In [4]:
import os

# List the contents of the unzipped directory
dataset_dir = 'content'  # Adjust based on where the files are
print(os.listdir(dataset_dir))

FileNotFoundError: [Errno 2] No such file or directory: 'content'

In [ ]:
import wandb

# Log in to WandB
wandb.login()

# Initialize a new WandB run
wandb.init(project='sign-language-detection')


wandb: Currently logged in as: shahprutha (shahprutha-svkm-s-narsee-monjee-institute-of-management-). Use `wandb login --relogin` to force relogin


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training and validation data
train_generator = datagen.flow_from_directory(
    'data',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    'data',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 33600 images belonging to 35 classes.
Found 8400 images belonging to 35 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Hyperparameters
config = wandb.config
config.epochs = 10
config.batch_size = 32

# Train the model
for epoch in range(config.epochs):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        epochs=1
    )

    # Log metrics
    wandb.log({
        'epoch': epoch + 1,
        'train_loss': history.history['loss'][0],
        'val_loss': history.history['val_loss'][0],
        'train_accuracy': history.history['accuracy'][0],
        'val_accuracy': history.history['val_accuracy'][0]
    })


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1050/1050 ━━━━━━━━━━━━━━━━━━━━ 227s 215ms/step - accuracy: 0.9455 - loss: 0.2202 - val_accuracy: 0.9925 - val_loss: 0.0274
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 226s 215ms/step - accuracy: 1.0000 - loss: 1.4721e-05 - val_accuracy: 0.9924 - val_loss: 0.0266
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - accuracy: 1.0000 - loss: 3.3763e-06

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

In [ ]:
wandb.finish()